In [49]:
!pip install pymorphy2 stop-words annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 83.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32896 sha256=56658bc2ae090f2462dbab0943e9c1035c58e60b2c38abfe7aa752ac4330504a
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552450 sha256=635a23a5ae978a9fa89a8c75d31f7271ccf09d965ec79bb5a9f39faf95481b62
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=2f636bebec

In [1]:
from google.colab import files
uploaded = files.upload()

Saving positive.csv to positive.csv


In [2]:
uploaded = files.upload()

Saving negative.csv to negative.csv


In [3]:
emoticon_dict = {
":)": "happy",
":-)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [4]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from collections import Counter
from string import punctuation
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [50]:
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
from tqdm import tqdm
from tqdm import tqdm_notebook

import gensim.models

In [5]:
stop_words = set(stopwords.words('russian'))
stop_words.add('это')

In [180]:
positive = pd.read_csv('/content/positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = 0
negative = pd.read_csv('/content/negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = 1
df = pd.concat([positive, negative], axis=0, ignore_index=True)

In [145]:
def emoji_to_regular_text(text):
    new_text = ' '.join(emoticon_dict[word] if word in emoticon_dict.keys() else word for word in text.split())
    return new_text
def remove_stop_words(text):
    new_text = [w for w in text.split() if w not in stop_words]
    return new_text
def lemm_my_text(text):
    new_text = [lemmatiser.lemmatize(w) for w in text]
    return new_text

lemmatiser = WordNetLemmatizer()

In [181]:
df['tweet_token'] = df.text.apply(lambda x: re.sub('@[\w]*', '', x))
df.tweet_token = df.tweet_token.apply(lambda x: x.lower())
df.tweet_token = df.tweet_token.apply(lambda x: emoji_to_regular_text(x))
df.tweet_token = df.tweet_token.apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
#df.tweet_token = df.tweet_token.apply(lambda x: re.sub(r'[^а-яА-Я0-9]', ' ', x))
df.tweet_token = df.tweet_token.apply(lambda x: re.sub(r'[^а-яА-Я]', ' ', x))
df.tweet_token = df.tweet_token.apply(lambda x: " ".join([w for w in x.split() if (len(w)>1) and len(w) < 11]))
#df['tweet_token'] = df.tweet_token.apply(lambda x: word_tokenize(x))
df['tweet_token'] = df.tweet_token.apply(lambda x: remove_stop_words(x))
df['tweet_token'] = df.tweet_token.apply(lambda x: lemm_my_text(x))
#df['tweet_token'] = df.tweet_token.apply(lambda x: " ".join(i for i in x))

In [182]:
df.head(5)

,text,label,tweet_token
0,"@first_timee хоть я и школота, но поверь, у на...",0,"[школота, поверь, самое, общество, предмет, типа]"
1,"Да, все-таки он немного похож на него. Но мой ...",0,"[таки, немного, похож, мальчик, равно]"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,0,"[идиотка, испугалась]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",0,"[углу, сидит, погибает, голода, ещ, порции, вз..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,0,"[значит, страшилка, блин, посмотрев, части, со..."


In [183]:
def preprocess_text(text):
  text2 = re.sub('@[\w]*', '', text)
  text2 = text2.lower()
  text2 = emoji_to_regular_text(text2)
  text2 = re.sub(r'[^\w\s]', ' ', text2)
  text2 = re.sub(r'[^а-яА-Я]', ' ', text2)
  text2 = " ".join([w for w in text2.split() if (len(w)>1) and len(w) < 11])
  #text2 = word_tokenize(text2)
  text2 = remove_stop_words(text2)
  text2 = lemm_my_text(text2)
  #text2 = " ".join(i for i in text2)
  return text2

In [184]:
preprocess_text(df['text'].values[0])

['школота', 'поверь', 'самое', 'общество', 'предмет', 'типа']

In [185]:
sentences = []

for line in tqdm_notebook(df.text):
    spls = preprocess_text(line)
    sentences.append(spls)

<ipython-input-185-d0067b3baada>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(df.text):


  0%|          | 0/226834 [00:00<?, ?it/s]

In [186]:
sentences[0]

['школота', 'поверь', 'самое', 'общество', 'предмет', 'типа']

In [208]:
window = 5
min_count = 5
vector_size = 300

In [209]:
modelW2V = Word2Vec(sentences=sentences, vector_size=vector_size, window=window, min_count=min_count)
modelFT = FastText(sentences=sentences, vector_size=vector_size,  window=window, min_count=min_count)

In [210]:
w2v_index = annoy.AnnoyIndex(vector_size ,'angular')
ft_index = annoy.AnnoyIndex(vector_size ,'angular')

index_map = {}
counter = 0

for line in tqdm_notebook(df.text):
        n_w2v = 0
        n_ft = 0
        spls = line
        index_map[counter] = spls
        question = preprocess_text(spls)


        vector_w2v = np.zeros(vector_size)
        vector_ft = np.zeros(vector_size)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)

        counter += 1


w2v_index.build(5)
ft_index.build(5)

<ipython-input-210-e3888426a9b9>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(df.text):


  0%|          | 0/226834 [00:00<?, ?it/s]

True

In [211]:
def get_response(question, index, model, index_map):
    question = preprocess_text(question)
    vector = np.zeros(vector_size)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    processed_answer = [index_map[i] for i in answers]
    #natural_answer = [df.loc[y[0]].tweet_token for y in [df.index[df['text'] == i].tolist() for i in processed_answer]]
    return processed_answer

In [214]:
df.text[0]

'@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)'

In [224]:
TEXT = 'хоть я и школота, самое :D общество профилирующий предмет типа)'

In [225]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)',
 '@Antindon делал то же самое. Получилось то же самое. Разобрал, сломал крепление шлейфа, теперь не работают некоторые кнопки((',
 '@mal_gr вот то же самое. у меня есть пара любимых магазинов, а другие не жалую х)',
 '@Shevchenkomax хотят самыми примитивными методами развалить Кавказ( самое страшное, что в это простой люд верит(((',
 'НТВ - это реальный, 100 % вредитель(((( Враг народа в буквальном смысле этого слова((( Самое плохое, что обыват… http://t.co/dpJOr6aq4U']

In [226]:
get_response(TEXT, ft_index, modelFT, index_map)

['@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)',
 '@PollyDoesntCare нееет((( мне сказали что самое главное это карта ...вопросы это так ...не самое важное ...ну и на крайняк вопрос можно сипи',
 '@official_tamae так что самое главное, их оргию снимает на камеру Тао ))))))',
 '-ВАХАХАХА СТИПЕНДИЯ,ОЛОЛО Я БОГАТ.\n-Слушай,тут зачет в пн,одолжи 2к,а?\n-Окау:(\nТо самое чувство.То.самое.чувство.',
 'знаете самое противное чувство?  это когда педали промокли насквозь , вот что сука противно)']